## Radiologists disagreements

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 8 radiologist predictions
rad1 = pd.read_csv('data/bc1_gt.csv')
rad2 = pd.read_csv('data/bc2_gt.csv')
rad3 = pd.read_csv('data/bc3_gt.csv')
rad4 = pd.read_csv('data/bc4.csv')
rad5 = pd.read_csv('data/bc5_gt.csv')
rad6 = pd.read_csv('data/bc6.csv')
rad7 = pd.read_csv('data/bc7_gt.csv')
rad8 = pd.read_csv('data/bc8.csv')

df_rad = rad1[['Study']].copy()
dfs = [rad1, rad2, rad3, rad5, rad7]
dfs = [rad1, rad2, rad3, rad4, rad5, rad6, rad7, rad8]
rad1.head()

,Study,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/test/patient64741/study1,0,0,0,0,1,0,1,1,0,0,0,0,0,1
1,CheXpert-v1.0/test/patient64742/study1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,CheXpert-v1.0/test/patient64743/study1,0,1,1,0,1,1,1,1,1,0,1,0,0,0
3,CheXpert-v1.0/test/patient64744/study1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,CheXpert-v1.0/test/patient64745/study1,0,0,0,1,1,0,1,0,1,0,1,0,0,0


In [3]:
df_rad = dfs[0][['Study']].copy()

for col in dfs[0].columns:
    if col == 'Study':
        continue
    col_preds = [df[col].values for df in dfs] 
    
    df_rad[col] = [list(row_preds) for row_preds in zip(*col_preds)]


In [4]:
def extract_patient_study(value):
    parts = value.split('/')
    patient = parts[2] 
    study = parts[3] 
    return pd.Series([patient, study])

df_rad[['patient', 'study']] = df_rad['Study'].apply(extract_patient_study)
df_rad.drop('Study', axis=1, inplace=True)


### Link with demographic information

In [5]:
df = pd.read_excel('data/chexpert_subgroups.xlsx')
csv_file = 'data/chexpert_subgroups.csv' 
df.to_csv(csv_file, index=False)
df_subgroups = pd.read_csv('data/chexpert_subgroups.csv')

In [6]:
df_subgroups.columns = ['patient', 'gender', 'age', 'race', 'etnicity']

In [7]:
df_subgroups_labels = pd.merge(df_rad, df_subgroups, on=['patient'], how='inner')

In [8]:
df_subgroups_labels = df_subgroups_labels.drop_duplicates(subset='patient', keep='first')
df_subgroups_labels = df_subgroups_labels.reset_index(drop=True)

In [9]:
df_subgroups_labels.head()

,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient,study,gender,age,race,etnicity
0,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]","[0, 1, 0, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]",patient64741,study1,Male,57,White,Non-Hispanic/Non-Latino
1,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 1, 0, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 1, 1]",patient64742,study1,Female,69,White,Non-Hispanic/Non-Latino
2,"[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 0, 1, 0, 1, 0]","[1, 0, 1, 1, 0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 1, 1, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0]","[0, 0, 1, 1, 0, 0, 1, 0]",patient64743,study1,Male,88,"White, non-Hispanic",Unknown
3,"[1, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 1, 1, 0, 0]","[0, 0, 1, 1, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 1, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 1, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 0]",patient64744,study1,Male,57,Other,Hispanic/Latino
4,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]","[0, 1, 0, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 1]","[1, 1, 1, 1, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 0, 1]","[0, 0, 1, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 1]",patient64745,study1,Male,92,White,Non-Hispanic/Non-Latino


### Disagreement analysis

In [10]:
diseases = ['Enlarged Cardiomediastinum', 'Cardiomegaly',  'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']

In [11]:
df_subgroups_labels['age'] = df_subgroups_labels['age'].apply(lambda x: 0 if x < 70 else 1)
df_subgroups_labels['race'] = df_subgroups_labels['race'].map({'White': 0, 'White, non-Hispanic': 0, 'Asian': 1, 'Black': 2})
df_subgroups_labels['sex'] = df_subgroups_labels['gender'].map({'Male': 0, 'Female': 1})

In [12]:
df = df_subgroups_labels.dropna()

In [13]:
df['race'] = df['race'].astype(int)

/tmp/ipykernel_749710/2602497011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].astype(int)


In [14]:
y_sex = df['sex']
y_race = df['race']
y_age = df['age']

### Dataset info

In [15]:
print(df['sex'].value_counts())
print(df['age'].value_counts())
print(df['race'].value_counts())


sex
0    202
1    125
Name: count, dtype: int64
age
0    196
1    131
Name: count, dtype: int64
race
0    276
1     51
Name: count, dtype: int64


### Disagreement according to sex, age and race

In [16]:

def disagreement(votes):
    """Fraction of raters not in the majority."""
    counts = pd.Series(votes).value_counts()
    maj = counts.max()
    return 1 - maj / len(votes)

overall = {}
for d in diseases:
    overall[d] = df[d].apply(disagreement).mean()

overall_df = pd.Series(overall).sort_values(ascending=False)
print("Overall radiologist disagreement (highest → lowest):")
print(overall_df)

subgroup_results = {}

df_sub = (
    df
    .groupby('sex')
    .apply(
        lambda grp: pd.Series({
            d: grp[d].apply(disagreement).mean() 
            for d in diseases
        })
    )
)
subgroup_results['sex'] = df_sub

print(f"\nDisagreement by sex:")
print(df_sub)

Overall radiologist disagreement (highest → lowest):
Enlarged Cardiomediastinum    0.189602
Cardiomegaly                  0.182722
Atelectasis                   0.176223
Lung Opacity                  0.135703
Edema                         0.116590
Pleural Effusion              0.113150
Consolidation                 0.094037
Support Devices               0.084862
Pneumonia                     0.048930
Pleural Other                 0.028287
Fracture                      0.019878
Lung Lesion                   0.018731
Pneumothorax                  0.011468
dtype: float64

Disagreement by sex:
     Enlarged Cardiomediastinum  Cardiomegaly  Lung Lesion  Lung Opacity  \
sex                                                                        
0                      0.189975      0.174505     0.017946      0.145421   
1                      0.189000      0.196000     0.020000      0.120000   

        Edema  Consolidation  Pneumonia  Atelectasis  Pneumothorax  \
sex                         

In [17]:
df_sub.mean(axis=1)

sex
0    0.093060
1    0.095154
dtype: float64

In [18]:
df_sub = (
    df
    .groupby('age')
    .apply(
        lambda grp: pd.Series({
            d: grp[d].apply(disagreement).mean() 
            for d in diseases
        })
    )
)
subgroup_results['age'] = df_sub

print(f"\nDisagreement by age:")
print(df_sub)


Disagreement by age:
     Enlarged Cardiomediastinum  Cardiomegaly  Lung Lesion  Lung Opacity  \
age                                                                        
0                      0.184949      0.176658     0.015944      0.127551   
1                      0.196565      0.191794     0.022901      0.147901   

        Edema  Consolidation  Pneumonia  Atelectasis  Pneumothorax  \
age                                                                  
0    0.096939       0.079082   0.045281     0.164541      0.014031   
1    0.145992       0.116412   0.054389     0.193702      0.007634   

     Pleural Effusion  Pleural Other  Fracture  Support Devices  
age                                                              
0            0.108418       0.023597  0.017857         0.078444  
1            0.120229       0.035305  0.022901         0.094466  


In [19]:
df_sub.mean(axis=1)

age
0    0.087176
1    0.103861
dtype: float64

In [20]:
df_sub = (
    df
    .groupby('race')
    .apply(
        lambda grp: pd.Series({
            d: grp[d].apply(disagreement).mean() 
            for d in diseases
        })
    )
)
subgroup_results['race'] = df_sub

print(f"\nDisagreement by race:")
print(df_sub)


Disagreement by race:
      Enlarged Cardiomediastinum  Cardiomegaly  Lung Lesion  Lung Opacity  \
race                                                                        
0                       0.191123      0.181612     0.017663      0.143116   
1                       0.181373      0.188725     0.024510      0.095588   

         Edema  Consolidation  Pneumonia  Atelectasis  Pneumothorax  \
race                                                                  
0     0.120924       0.089674   0.043931     0.180707      0.009511   
1     0.093137       0.117647   0.075980     0.151961      0.022059   

      Pleural Effusion  Pleural Other  Fracture  Support Devices  
race                                                              
0             0.118207       0.029891  0.022645         0.086051  
1             0.085784       0.019608  0.004902         0.078431  


In [21]:
df_sub.mean(axis=1)

race
0    0.095004
1    0.087670
dtype: float64